In [30]:
from sdRDM.generator import generate_python_api


generate_python_api("../specifications/model.md", "../", "pyeed")

<unknown>:85: SyntaxWarning: invalid escape sequence '\d'


In [1]:
%reload_ext autoreload
%autoreload 2

from pyeed.core.proteinsequence import ProteinSequence

In [30]:
seq = ProteinSequence.from_ncbi("XP_017143456.1")

yes


In [31]:
print(seq)

ProteinSequence
├── id = XP_017143456.1
├── name = fructose-bisphosphate aldolase isoform X3 [Drosophila miranda]
├── sequence = MTTYFNYPSKELQDELRGIAQAIVAPGKGILAADESVSTMGKRLQDIGVENSDENRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFVDILKKKGIIPGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPQEIGLATVLALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLGRPWALTFSYGRALQASVLKAWGGKKENIAAGQNELLKRARANALASQGKYVAGSIPSYAANTSLFIAGHKY
├── organism
│   └── Organism
│       ├── id = taxon:7229
│       ├── name = Drosophila miranda
│       └── taxonomy_id = 7229
├── regions
│   └── 0
│       └── Region
│           ├── id = region121
│           ├── start = 14
│           ├── end = 363
│           ├── note = Fructose-bisphosphate aldolase class-I; pfam00274
│           ├── name = Glycolytic
│           └── cross_reference = CDD:425574
├── sites
│   ├── 0
│   │   └── Site
│   │       ├── id = site363
│   │       ├── name = active
│   │  

In [21]:
res = seq.blast(n_hits=100)

In [33]:
res

[ProteinSequence(id='AEB39622.1', name='FI14722p [Drosophila melanogaster]', sequence='MTLIASEGIIAEVSVTLLSYKNIIVENYTRISKMTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY', organism=Organism(id='taxon:7227', name='Drosophila melanogaster', taxonomy_id='7227'), regions=[Region(id='region18', start=47, end=394, note='Fructose-bisphosphate aldolase class-I; pfam00274', name='Glycolytic', cross_reference='CDD:425574')], sites=[Site(id='site54', name='active', type='active', positions=[66, 67, 68, 71, 140, 179, 181, 220, 262, 302, 303, 304, 332, 334, 335], cross_reference='CDD:188635'), Site(id='site55', name='intersubunit interface [polypeptide binding]', type='other', positions=[143,

### Building the SQL database

Next, we are going to use the ```build_sql_database``` function to set up an SQLite database file. This one will be used later on to populate data from our model. The database will construct a tabe for each object/attribute and thus facilitate an easy transfer from an application to the database.

### Creating a dataset

In order to demonstrate how to populate the database using our model, we are going to construct a small dataset using the data model we just loaded.

### Populating the database

Finally, we are going to add the data to our previously created database by using the datasets ```to_sql``` method where we also submit the location of our database file. This will create an Object Relation Model (ORM) that represents the Database structure and map the values present in our dataset to the corresponding tables.

In [ ]:
from sdRDM import DataModel
from sdRDM_db import DBConnector, create_tables

In [ ]:
# Establish a connection to the database
db = DBConnector(
    username="root", password="root", host="localhost", db_name="ExampleDB"
)

In [ ]:
db

DBConnector(host='localhost', username='root', password='root', db_name='ExampleDB', port=5432, dbtype=<SupportedBackends.POSTGRES: 'postgres'>, connection=<ibis.backends.postgres.Backend object at 0x14cbe0a10>)

In [ ]:
db.connection.list_tables()

OperationalError: (psycopg2.OperationalError) connection to server on socket "/tmp/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

DBConnector(host='localhost', username='root', password='root', db_name='ExampleDB', port=5432, dbtype=<SupportedBackends.POSTGRES: 'postgres'>, connection=<ibis.backends.postgres.Backend object at 0x173c9c510>)